In [1]:
"""
Contains various recommondation implementations
all algorithms return a list of movieids
"""

import pandas as pd
import numpy as np
from utils import lookup_movieId, match_movie_title

In [2]:
user_rating = {
    'the lion king': 5,
    'terminator': 5,
    'star wars': 2
}

In [ ]:
print

In [3]:
user_mat = pd.read_csv('data/cleaned_user_item_matrix.csv', index_col=[0])

In [4]:
dictionary = pd.read_csv('data/cleaned_movies_dictionary.csv', index_col=[0])

In [5]:
movies = pd.read_csv('data/movies_clusters_ratings.csv', index_col='movieid')  

In [8]:
def unseen_movies(movies, user_rating):
    """
    return list of all unseen movies by selecting the movies without rating by user 
    """
    user = pd.DataFrame(user_rating, index=[0])
    user_t = user.T.reset_index()
    user_movie_entries = list(user_t["index"])
    movie_titles = list(movies["title"])
    parsed_title = [match_movie_title(title, movie_titles) for title in user_movie_entries]
    
    unseen_movies = movies.copy()
    unseen_movies = unseen_movies.reset_index()
    unseen_movies = unseen_movies.set_index("title")
    unseen_movies.drop(parsed_title, inplace=True)
    
    return unseen_movies

In [9]:
unseen_movies(movies, user_rating)

,movieid,cluster_no,rating
title,,,
Toy Story,1,11,3.92
Jumanji,2,11,3.43
Grumpier Old Men,3,21,3.26
Waiting to Exhale,4,10,2.36
Father of the Bride Part II,5,1,3.07
...,...,...,...
Black Butler: Book of the Atlantic,193581,2,4.00
No Game No Life: Zero,193583,2,3.50
Flint,193585,0,3.50


In [7]:
def recommend_random(movies, user_rating, k=5):
    """
    return k random unseen movies for user 
    """
    recommend = unseen_movies(movies, user_rating)
    random_movies = np.random.choice(list(recommend.index), replace=False, size=k)
    
    return random_movies

In [8]:
def get_popularity(user_mat, dictionary):
    user_mat = user_mat.T
    user_mat = user_mat.astype(float)
    
    null_count = []
    for i in user_mat.index:
        count = user_mat.loc[[i]].isna().sum().sum()
        null_count.append(count)
    
    user_mat['null_count'] = null_count
    user_mat['total_top_rating'] = user_mat.select_dtypes(np.number).gt(3.5).sum(axis=1)
    popular = user_mat[['null_count','total_top_rating']]
    popular = popular.reset_index()
    popular['movieid'] = popular['index'].astype(int)
    popular.set_index('movieid', inplace=True)
    movies_popularity = popular.join(dictionary)
    movies_popularity = movies_popularity.drop('index', axis=1)
    
    return movies_popularity

In [9]:
def recommend_most_popular(user_rating, movies, k=5):
    """
    return k movies from list of 50 best rated movies unseen for user
    """
    popularity = get_popularity(user_mat, dictionary)
    movies = movies.join(popularity[['null_count','total_top_rating']])
    recommend = unseen_movies(movies, user_rating)
    # To get the most popular, you need to join the table with user ids and then check the movie that has been rated by most users!
    
    most_popular = list(recommend.sort_values('total_top_rating', ascending=False).head(k).index)

    return most_popular

In [10]:
recommend_most_popular(user_rating, movies)

['Shawshank Redemption, The',
 'Forrest Gump',
 'Pulp Fiction',
 'Silence of the Lambs, The',
 'Matrix, The']

In [11]:
def get_cluster(movies, user_rating):
    """
    return list of all unseen movies by selecting the movies without rating by user 
    """
    user = pd.DataFrame(user_rating, index=[0])
    user_t = user.T.reset_index()
    user_movie_entries = list(user_t["index"])
    movie_titles = list(movies["title"])
    parsed_title = [match_movie_title(title, movie_titles) for title in user_movie_entries]
    user_cluster = list(movies['cluster_no'].loc[movies.apply(lambda x: x.title in parsed_title, axis=1)])
    
    return user_cluster

In [12]:
def recommend_from_same_cluster(user_rating, movies, k=3):
    user_cluster = get_cluster(movies, user_rating)
    popularity = get_popularity(user_mat, dictionary)
    movies = movies.join(popularity[['null_count','total_top_rating']])
    unseen = unseen_movies(movies, user_rating)
    movies_same_cluster = unseen.loc[unseen.apply(lambda x: x.cluster_no in user_cluster, axis=1)]
    
    top_20 = movies_same_cluster.sort_values('total_top_rating',ascending = False).groupby('cluster_no').head(20)
    recommend_2 = top_20.sample(frac = 1.0).groupby('cluster_no').head(5)
    recommend_cluster = list(recommend_2.index.values)
            
    return recommend_cluster

In [13]:
recommend_from_same_cluster(user_rating, movies)

['Wizard of Oz, The',
 'Star Wars: Episode VI - Return of the Jedi',
 'Monsters, Inc.',
 'Guardians of the Galaxy',
 '28 Days Later',
 'Avatar',
 'Star Wars: Episode III - Revenge of the Sith',
 'Spirited Away (Sen to Chihiro no kamikakushi)',
 'Incredibles, The',
 'Toy Story']

In [14]:
from sklearn.impute import SimpleImputer

In [15]:
from sklearn.decomposition import NMF

In [16]:
user_mat

,1,2,3,4,5,6,915,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# process user-item matrix
user_r = user_mat.T.reset_index()
user_r['movieid'] = user_r['index'].astype(int)
user_r.set_index('movieid', inplace=True)
user_title = user_r.join(dictionary)
user_title.set_index('title', inplace=True)
user_title.drop('index', axis=1, inplace=True)

# syn user rating with user-item matrix
user_df = pd.DataFrame(user_rating, index=[0])
user_t = user_df.T.reset_index()
user_movie_entries = list(user_t['index'])
movie_titles = list(user_title.index.values)
parsed_title = [match_movie_title(title, movie_titles) for title in user_movie_entries]
user_df.columns = parsed_title
user_df = user_df.T
user_df.rename_axis(index='title', inplace=True)

# remove null values
all_user_info = user_title.join(user_df)
user = all_user_info[0]
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(all_user_info.T)
impute_transform = imputer.transform(all_user_info.T)
new_df = pd.DataFrame(impute_transform.T, columns=all_user_info.columns, index=all_user_info.index.values)

# run nmf
nmf = NMF(n_components=20, max_iter=1000)
nmf.fit(new_df)
P = nmf.transform(new_df)
Q = nmf.components_
Q_df = pd.DataFrame(Q, columns=new_df.columns)
PQ = P.dot(Q)
PQ_df = pd.DataFrame(PQ, columns=new_df.columns, index=new_df.index.values)

#recommendation
top_20 = PQ_df[0].sort_values(ascending = False).head(20)
recommend_5 = top_20.sample(frac = 1.0).head(5)
recommend_nmf = list(recommend_5.index.values)


/Users/dilaraarif/miniforge3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/dilaraarif/miniforge3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn(


In [35]:
user_df

,0
title,
"Lion King, The",5
Terminator 2: Judgment Day,5
Star Wars: Episode IV - A New Hope,2


In [53]:
mask = ~PQ_df.index.isin(user_df.index)
result = PQ_df.loc[mask]
print(result)

                                         1         2         3         4    \
Toy Story                           4.083697  3.993538  4.007843  3.768279   
Jumanji                             3.868514  3.973328  3.974715  3.801500   
Grumpier Old Men                    3.218492  2.988176  2.967845  2.957907   
Waiting to Exhale                   3.025198  2.998282  2.976074  2.980134   
Father of the Bride Part II         3.047373  3.003795  2.976392  2.963525   
...                                      ...       ...       ...       ...   
Black Butler: Book of the Atlantic  3.996195  4.000826  4.000159  4.002619   
No Game No Life: Zero               3.496670  3.500723  3.500139  3.502292   
Flint                               3.496670  3.500723  3.500139  3.502292   
Bungo Stray Dogs: Dead Apple        3.496670  3.500723  3.500139  3.502292   
Andrew Dice Clay: Dice Rules        3.996195  4.000826  4.000159  4.002619   

                                         5         6         7 

In [57]:
def recommend_with_NMF(user_mat, user_rating, dictionary, k=5):
    """
    NMF Recommender
    INPUT
    - user_vector with shape (1, #number of movies)
    - user_item_matrix
    - trained NMF model

    OUTPUT
    - a list of movieIds
    """
    # process user-item matrix
    user_r = user_mat.T.reset_index()
    user_r['movieid'] = user_r['index'].astype(int)
    user_r.set_index('movieid', inplace=True)
    user_title = user_r.join(dictionary)
    user_title.set_index('title', inplace=True)
    user_title.drop('index', axis=1, inplace=True)

    # syn user rating with user-item matrix
    user_df = pd.DataFrame(user_rating, index=[0])
    user_t = user_df.T.reset_index()
    user_movie_entries = list(user_t['index'])
    movie_titles = list(user_title.index.values)
    parsed_title = [match_movie_title(title, movie_titles) for title in user_movie_entries]
    user_df.columns = parsed_title
    user_df = user_df.T
    user_df.rename_axis(index='title', inplace=True)

    # remove null values
    all_user_info = user_title.join(user_df)
    user = all_user_info[0]
    imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imputer.fit(all_user_info.T)
    impute_transform = imputer.transform(all_user_info.T)
    new_df = pd.DataFrame(impute_transform.T, columns=all_user_info.columns, index=all_user_info.index.values)

    # run nmf
    nmf = NMF(n_components=5, max_iter=1000)
    nmf.fit(new_df)
    P = nmf.transform(new_df)
    Q = nmf.components_
    Q_df = pd.DataFrame(Q, columns=new_df.columns)
    PQ = P.dot(Q)
    PQ_df = pd.DataFrame(PQ, columns=new_df.columns, index=new_df.index.values)

    #recommendation
    mask = ~PQ_df.index.isin(user_df.index)
    result = PQ_df.loc[mask]
    top_50 = PQ_df[0].sort_values(ascending = False).head(50)
    recommend_5 = top_50.sample(frac = 1.0).head(5)
    recommend_nmf = list(recommend_5.index.values)
    
    return recommend_nmf

In [61]:
recommend_with_NMF(user_mat, user_rating, dictionary, k=5)

/Users/dilaraarif/miniforge3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/dilaraarif/miniforge3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn(


['Lincoln',
 'Rosencrantz and Guildenstern Are Dead',
 'Raging Bull',
 'Hedwig and the Angry Inch',
 'Reign Over Me']